# [Going Deeper NLP 08] 한영 번역기 만들기
attention을 기반으로 한 seq2seq모델로 한영 번역기를 생성하는 프로젝트  

---

## 프로젝트 목표
---
- seq2seq 모델을 구현하며 이해한다.
- 기대 효과를 충족하는 모델을 구현한다.

## 프로젝트 설명
---
1. 데이터 다운로드 : [링크](https://github.com/jungyeul/korean-parallel-corpora/blob/master/korean-english-news-v1/korean-english-park.train.tar.gz)
2. 데이터 정제 
- 중복, 결측치 제거
- 한글 데이터 처리
- 토큰 추가 
3. 데이터 토큰화 
4. 모델 설계 
5. 훈련 및 attention map 시각화 

    
## 0. Import module, library

In [1]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.1.0-rc2


## 1. 데이터 전처리 
- 중복 데이터 제거
- Input, output 데이터의 쌍을 유지해야한다. 

In [6]:
file_path = os.getenv('HOME')+'/aiffel/s2s_translation'
print(file_path)

/home/aiffel-dj26/aiffel/s2s_translation


In [8]:
with open(file_path+'/korean-english-park.train.ko', "r") as f:
    raw_ko = f.read().splitlines()

print("Data Size:", len(raw_ko))
print("Example:")

for sen in raw_ko[0:100][::20]: print(">>", sen)

Data Size: 94123
Example:
>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>> 북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.
>> "경호 로보트가 침입자나 화재를 탐지하기 위해서 개인적으로, 그리고 전문적으로 사용되고 있습니다."
>> 수자원부 당국은 논란이 되고 있고, 막대한 비용이 드는 이 사업에 대해 내년에 건설을 시작할 계획이다.
>> 또한 근력 운동은 활발하게 걷는 것이나 최소한 20분 동안 뛰는 것과 같은 유산소 활동에서 얻는 운동 효과를 심장과 폐에 주지 않기 때문에, 연구학자들은 근력 운동이 심장에 큰 영향을 미치는지 여부에 대해 논쟁을 해왔다.


In [9]:
with open(file_path+'/korean-english-park.train.en', "r") as f:
    raw_en = f.read().splitlines()

print("Data Size:", len(raw_en))
print("Example:")

for sen in raw_en[0:100][::20]: print(">>", sen)

Data Size: 94123
Example:
>> Much of personal computing is about "can you top this?"
>> Amid mounting pressure on North Korea to abandon its nuclear weapons program Japanese and North Korean diplomats have resumed talks on normalizing diplomatic relations.
>> “Guard robots are used privately and professionally to detect intruders or fire,” Karlsson said.
>> Authorities from the Water Resources Ministry plan to begin construction next year on the controversial and hugely expensive project.
>> Researchers also have debated whether weight-training has a big impact on the heart, since it does not give the heart and lungs the kind of workout they get from aerobic activities such as brisk walking or running for at least 20 minutes.


# 루브릭 평가
---
1. **번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.**   

    - 구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.


2. **Attentional Seq2seq 모델이 정상적으로 구동된다.**

    - seq2seq 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.
     
3. **테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.** 
 
     - 테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.

    
# 회고
---

## 어려웠던 부분 
- 수식을 이해하는 부분이 어려웠으나 퍼실 @홍진표님의 도움으로 잘 이해하고 넘어갈 수 있었다.

## 알아낸 점 혹은 모호한 부분 
- 데이터의 편향이 있을 수 있다라는 점을 알고 있었는데 이를 평가할 수 있는 지표가 있다는 점을 배울 수 있었다. 
- 이런 편향에 대해 어떠한 처리가 필요한 것인지 궁금하지만 이는 임베딩을 만드는 모델에서 처리해야할 것 같다. 
- 또한 편향이 데이터의 특성이 될 수 있는데, 편향을 없애는 것외에 편향이 있다는 사실을 모델에 적용할 수 있는 방법에 대해 고민이 된다.
- 행렬 계산을 통해 여러 attribute에 대한 WEAT score를 한 번에 계산할 수 있었다. 
- 예술영화와 일반영화에 대한 구분 기준이 모호하다고 생각한다. 두 구분은 영화 장르에 따라 구분되지 않으며, 이는 두 구분을 대표하는 단어를 보면 중복되는 단어가 많으며 기준이 모호하다는 것을 볼 수 있다. 
- 더 구분 기준이 뚜렷한 예시로 프로젝트를 진행했으면 결과에 대해 납득을 할 수 있었을 것이다. 

## 느낀 점 
- 지표를 공부하고 평가하는 방법을 배웠으니 이를 어디에 활용하면 더 좋은 모델을 구성할 수 있을지 고민하고 싶다. 